In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [2]:
!kaggle datasets download -d gpiosenka/cats-in-the-wild-image-classification

100% 123M/123M [00:06<00:00, 25.0MB/s]
100% 123M/123M [00:06<00:00, 19.8MB/s]


In [3]:
!unzip -q cats-in-the-wild-image-classification.zip

In [15]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential, load_model
from keras.layers import Input, Dense, Activation, Flatten, Conv2D, MaxPool2D, Dropout, GlobalAveragePooling2D, BatchNormalization
from keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import numpy as np

In [29]:
img_width, img_height = 224, 224
EPOCHS = 25
Batch_size = 32

train_dir = "train/"
test_dir ="test/"
val_dir = "valid/"

In [23]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                            shear_range=0.2,
                            zoom_range=0.2, 
                            rotation_range=0.2,
                            height_shift_range=0.2,
                            width_shift_range=0.2,
                            horizontal_flip=True, 
                            fill_mode="nearest")
test_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)


train_data = train_datagen.flow_from_directory(train_dir,
                                       target_size=(img_width, img_height),
                                       shuffle=True)

test_data = test_datagen.flow_from_directory(test_dir,
                                      target_size=(img_width, img_height),
                                      shuffle=False)

valid_data = validation_datagen.flow_from_directory(val_dir,
                                    target_size=(img_width, img_height),
                                    shuffle=True)

train_data.class_indices

Found 2339 images belonging to 10 classes.
Found 50 images belonging to 10 classes.
Found 50 images belonging to 10 classes.


{'AFRICAN LEOPARD': 0,
 'CARACAL': 1,
 'CHEETAH': 2,
 'CLOUDED LEOPARD': 3,
 'JAGUAR': 4,
 'LIONS': 5,
 'OCELOT': 6,
 'PUMA': 7,
 'SNOW LEOPARD': 8,
 'TIGER': 9}

In [25]:
res = tf.keras.applications.ResNet50V2(
    input_shape=(224,224,3),
    include_top=False,
)
res.trainable = False

94668760/94668760 [==============================] - 5s 0us/step


In [26]:
res_model = Sequential()
res_model.add(res)
res_model.add(Dropout(0.25))
res_model.add(GlobalAveragePooling2D())
res_model.add(Flatten())
res_model.add(Dense(256, activation='relu'))
res_model.add(BatchNormalization())
res_model.add(Dropout(0.5))
res_model.add(Dense(10, activation='softmax'))
res_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 7, 7, 2048)        23564800  
                                                                 
 dropout_7 (Dropout)         (None, 7, 7, 2048)        0         
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 flatten_3 (Flatten)         (None, 2048)              0         
                                                                 
 dense_5 (Dense)             (None, 256)               524544    
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                        

In [27]:
print("[INFO] compiling model...")
opt = Adam(learning_rate=1e-4)
res_model.compile(loss="categorical_crossentropy", 
              optimizer=opt,
              metrics=["accuracy"])

my_callbacks = [EarlyStopping(monitor="val_loss", 
                              patience=5, 
                              verbose=1, 
                              mode="auto"),
                
                ModelCheckpoint(filepath="./models/best_resnetmodel_BIGCATS.h5",
                                monitor= 'val_loss', 
                                verbose= 1,
                                save_best_only= True, 
                                mode = 'auto')
               ]

[INFO] compiling model...


In [30]:
print("[INFO] training ...")
hist = res_model.fit(train_data, 
                     batch_size=Batch_size,
                     steps_per_epoch=train_data.samples // Batch_size,
                     validation_steps=valid_data.samples // Batch_size,
                     epochs=EPOCHS,
                     callbacks=my_callbacks,
                     validation_data=valid_data
                    )

[INFO] training ...
Epoch 1/25
73/73 [==============================] - ETA: 0s - loss: 0.5529 - accuracy: 0.8153
Epoch 1: val_loss improved from 0.51825 to 0.20656, saving model to ./models/best_resnetmodel_BIGCATS.h5
73/73 [==============================] - 31s 420ms/step - loss: 0.5529 - accuracy: 0.8153 - val_loss: 0.2066 - val_accuracy: 0.8750
Epoch 2/25
73/73 [==============================] - ETA: 0s - loss: 0.5089 - accuracy: 0.8401
Epoch 2: val_loss improved from 0.20656 to 0.14279, saving model to ./models/best_resnetmodel_BIGCATS.h5
73/73 [==============================] - 30s 417ms/step - loss: 0.5089 - accuracy: 0.8401 - val_loss: 0.1428 - val_accuracy: 0.9375
Epoch 3/25
73/73 [==============================] - ETA: 0s - loss: 0.4055 - accuracy: 0.8713
Epoch 3: val_loss did not improve from 0.14279
73/73 [==============================] - 31s 420ms/step - loss: 0.4055 - accuracy: 0.8713 - val_loss: 0.2168 - val_accuracy: 0.8750
Epoch 4/25
73/73 [===========================

In [32]:
res_model = load_model("models/best_resnetmodel_BIGCATS.h5")
res_model.evaluate(test_data)

2/2 [==============================] - 37s 86ms/step - loss: 0.0418 - accuracy: 1.0000


[0.0417960062623024, 1.0]